## Extract data from database
I only extract relations whose name starts with course or users according to Miguel's suggestion.

In [1]:
import psycopg2
import psycopg2.extras

database configuration

In [2]:
dbname = "pps_prd"
user = "ppsuser"
password = "jF2RwCuTgbXrKMb]$fxUVCpGD9Aw"
host = "machine-learning.cq0an2uihaqt.us-east-1.rds.amazonaws.com"

In [24]:

# exercise
conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

#cursor.execute("SELECT EXISTS(SELECT table_name FROM information_schema.tables WHERE table_type='BASE TABLE' AND table_name!~ '^(pg_|sql_)');")
cursor.execute("select column_name from information_schema.columns where table_name = 'auth_group_permissions';")
column_names = [row[0] for row in cursor]
#cursor.execute("select relname,reltuples from pg_class where relkind='r' and relname = 'auth_group_permissions';")
#m=cursor.fetchone()
cursor.close()
conn.close()
print column_names

['id', 'group_id', 'permission_id']


In [21]:
def copyToFile():
    tableList = []
    count = 0
    conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
    curSelectTable = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    curSelectTable.execute("select relname,reltuples from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
    for table in curSelectTable.fetchall():
        if(table[1]>0):
            tName = str(table[0])
            #curCheckNull = conn.cursor()
            #queryCheckNull = "SELECT EXISTS(SELECT 1 FROM pg_class WHERE relname=\'"+table+"\');"
            #curCheckNull.execute(queryCheckNull)
            #if curCheckNull.fetchone()[0] == True:
            if("course" in tName or "user" in tName):
                count += 1
                tableList.append(tName)
                curCopyTo = conn.cursor()
                filePath = "data/"+tName+".csv" #need change the path
                queryCopy = "COPY %s TO STDOUT WITH CSV HEADER DELIMITER AS ',';"
                with open(filePath, 'w') as f:
                    curCopyTo.copy_expert(sql=queryCopy % tName,file=f)
                    print tName +" is done."
    conn.close()
    print count, " files are copied."
    

In [22]:
copyToFile()

course_unit_articles is done.
course_resource is done.
course_unit_resources is done.
course_classroom is done.
course_article_questions is done.
course_course is done.
user_app_address is done.
user_app_customuser_user_permissions is done.
user_app_customuser is done.
user_messages_room_subscribers is done.
course_video is done.
course_article is done.
user_messages_message is done.
course_course_units is done.
user_messages_room_new_message is done.
user_messages_room is done.
course_progress is done.
course_unit is done.
course_match is done.
course_subquestionresponse is done.
course_match_student_match is done.
course_questionresponse_new_subresponse is done.
course_question is done.
course_questionresponse_new_response is done.
course_questionresponse is done.
25  files are copied.


Totally 25 files are copied to local.

In [20]:
# exercise for copy expert
SQL_STATEMENT = "COPY course_question TO STDOUT WITH CSV HEADER DELIMITER AS ',';"
conn = psycopg2.connect(dbname = dbname,user = user, password = password, host = host)
cursor = conn.cursor()
tName='course_question'
with open("course_question.csv", 'w+') as f:
    cursor.copy_expert(sql=SQL_STATEMENT, file=f)
    #cursor.copy_to(f,tName)
cursor.close()